In [1]:
#|default_exp datadownload

In [45]:
from pathlib import Path
import requests
import csv
from io import StringIO
from rich import inspect
import pandas as pd
import json

# Data download

In [55]:
def extract_metadata(metadata_url):
    """
    Extracts a list of filenames and urls from Open Cananda metadata url.

    Parameters
    ----------
    metadata_url : str
        Fuel consumption ratings metadata url from Open Canada website.

    Returns
    -------
    english_resources_df : pd.DataFrame
        DataFrame of file names and urls for energy consumption ratings.
    """
    try:      
        metadata_resp = requests.get(metadata_url)
    except requests.exceptions.RequestException as e:
        # If request fails, return an error message and stop.
        print(f'Error making url request: {e}')
    
    try:     
        metadata_json = metadata_resp.json()
    except json.JSONDecodeError:
        # If parsing json fails, return an error message and stop.
        print(f'Error: Response is not valid json')
        
    # Access list of downloadable resources
    resources_df = pd.DataFrame(metadata_json['result']['resources'])

    # Change language coding and extract English only resources
    resources_df['language'] = resources_df['language'].apply(lambda item : item[0])
    english_resources_df = resources_df[resources_df['language'] == 'en']
    
    return english_resources_df[['name', 'url']]

In [4]:
#|export
def extract_raw_data(url, file_name):
    """
    Extract raw data from a URL

    Parameters
    ----------
    url : str
        URL to extract data from
        
    file_name : str or Path object
        file name for raw data dump
        
    """
    
    try:
        # Request data from url
        response = requests.get(url)
        content_type = response.headers['content-type']
        response_text = response.text
        print(f'Response status: {response.status_code}\nContent Type: {content_type}')

        # Save request content to csv file
        with open(file_name, mode='w', newline='') as csvfile:
            csvfile.write(response_text)

        print(f'csv file: {file_name} saved')
        
    # Catch errors    
    except requests.exceptions.HTTPError as err_h:
        print(f'HTTP error occured:{err_h}')
    except requests.exceptions.ConnectionError as err_c:
        print(f'Error connecting:{err_c}')
    except requests.exceptions.Timeout as err_t:
        print(f'Timeout Error:{err_t}')
    except requests.exceptions.RequestException as err:
        print(f'There was an unknown error:{err}')

In [5]:
def merge_top_two_rows(input_file, output_file):
    # Open the input CSV file for reading
    with open(input_file, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        
        # Read the first two rows from the input file
        header_row = next(reader)
        second_row = next(reader)
        
        # Merge the two rows into one header
        merged_header = [f"{header_row[i]} {second_row[i]}" for i in range(len(header_row))]
        
        # Open the output CSV file for writing
        with open(output_file, mode='w', newline='') as output_csvfile:
            writer = csv.writer(output_csvfile)
            
            # Write the merged header to the output file
            writer.writerow(merged_header)
            
            # Copy the rest of the rows from the input file to the output file
            for row in reader:
                writer.writerow(row)

In [36]:
def rename_columns(df):
    """
    Renames DataFrame columns by removing unessessary whitepace, replacing spaces with _ and transforming to lowercase.

    Parameters
    ----------
    df:
        DataFrame with columns to clean

    """
    # Drop empty columns and rows from the DataFrame
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, thresh=5, inplace=True)

    # Remove whitespace, replace spaces with _ and change to lower case
    for col in df.columns:
        cleaned_col = col.strip().replace(" ", "_").lower()
        df.rename(columns={col: cleaned_col}, inplace=True)
    return df

In [7]:
def clean_dataframe(df):
    """
    Clean vehicle fuel ratings DataFrame.

    Assumes DataFrame has already been passed to rename_columns()

    """
    pass

## Script

In [42]:
url = 'https://natural-resources.canada.ca/sites/nrcan/files/oee/files/csv/MY2023%20Fuel%20Consumption%20Ratings.csv'
metadata_url = 'https://open.canada.ca/data/api/action/package_show?id=98f1a129-f628-4ce4-b24d-6f16bf24dd64'

In [43]:
r = requests.get(url)

In [ ]:
meta_url = 'https://open.canada.ca/data/api/action/package_show?id=98f1a129-f628-4ce4-b24d-6f16bf24dd64'

metadata_resp = requests.get(meta_url)

metadata_json = metadata_resp.json()

resources_df = pd.DataFrame(metadata_json['result']['resources'])

resources_df.head()

resources_df['language'] = resources_df['language'].apply(lambda item : item[0])

english_resources_df = resources_df[resources_df['language'] == 'en']

english_resources_df[['name', 'url']]

In [44]:
metadata_resp.headers

{'Server': 'nginx', 'Date': 'Mon, 25 Sep 2023 16:00:55 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '40666', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=5', 'Cache-Control': 'public, must-revalidate, max-age=0', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'}

In [38]:
# Download meta data and use JSON normalise.

In [53]:
inspect(r.headers, help=True)

╭─────────────────────────────── <class 'requests.structures.CaseInsensitiveDict'> ───────────────────────────────╮
│ A case-insensitive ``dict``-like object.                                                                        │
│                                                                                                                 │
│ Implements all methods and operations of                                                                        │
│ ``MutableMapping`` as well as dict's ``copy``. Also                                                             │
│ provides ``lower_items``.                                                                                       │
│                                                                                                                 │
│ All keys are expected to be strings. The structure remembers the                                                │
│ case of the last key to be set, and ``iter(instance)``,                                                         │
│ ``keys()``, ``items()``, ``iterkeys()``, and ``iteritems()``                                                    │
│ will contain case-sensitive keys. However, querying and contains                                                │
│ testing is case insensitive::                                                                                   │
│                                                                                                                 │
│     cid = CaseInsensitiveDict()                                                                                 │
│     cid['Accept'] = 'application/json'                                                                          │
│     cid['aCCEPT'] == 'application/json'  # True                                                                 │
│     list(cid) == ['Accept']  # True                                                                             │
│                                                                                                                 │
│ For example, ``headers['content-encoding']`` will return the                                                    │
│ value of a ``'Content-Encoding'`` response header, regardless                                                   │
│ of how the header name was originally stored.                                                                   │
│                                                                                                                 │
│ If the constructor, ``.update``, or equality comparison                                                         │
│ operations are given keys that have equal ``.lower()``s, the                                                    │
│ behavior is undefined.                                                                                          │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {'Content-Type': 'text/csv', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sun, 17   │ │
│ │ Sep 2023 09:40:10 GMT', 'Server': 'Apache/2.4.37 (Red Hat Enterprise Linux) OpenSSL/1.1.1k',                │ │
│ │ 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'X-Content-Type-Options':      │ │
│ │ 'nosniff', 'Last-Modified': 'Wed, 23 Aug 2023 17:20:40 GMT', 'ETag': 'W/"1a1301-6039a54adba27"',            │ │
│ │ 'Cache-Control': 'max-age=1209600', 'Expires': 'Sun, 01 Oct 2023 09:40:10 GMT', 'Content-Encoding': 'br',   │ │
│ │ 'Vary': 'Accept-Encoding', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1                                    │ │
│ │ 72284d4e8970ebb6888dfd7f2b2b621c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LHR62-C3', 'X-Amz-Cf-Id':   │ │
│ │ 'F0HlP-41l6VG7OMYgyABTxhlGLGQq-2qhxO7kF2jlSk1trc3KEQ

In [51]:
inspect(requests.RequestException, help=True, all=True)

╭──────────────────────────────── <class 'requests.exceptions.RequestException'> ─────────────────────────────────╮
│ class RequestException(*args, **kwargs):                                                                        │
│                                                                                                                 │
│ There was an ambiguous exception that occurred while handling your                                              │
│ request.                                                                                                        │
│                                                                                                                 │
│                 args = <attribute 'args' of 'BaseException' objects>                                            │
│            __cause__ = <attribute '__cause__' of 'BaseException' objects>                                       │
│   characters_written = <attribute 'characters_written' of 'OSError' objects>                                    │
│          __context__ = <attribute '__context__' of 'BaseException' objects>                                     │
│             __dict__ = mappingproxy({                                                                           │
│                            '__module__': 'requests.exceptions',                                                 │
│                            '__doc__': 'There was an ambiguous exception that occurred while handling your\n     │
│                        request.\n    ',                                                                         │
│                            '__init__': <function RequestException.__init__ at 0x7f0748347ba0>,                  │
│                            '__weakref__': <attribute '__weakref__' of 'RequestException' objects>               │
│                        })                                                                                       │
│              __doc__ = 'There was an ambiguous exception that occurred while handling your\n    request.\n    ' │
│                errno = <member 'errno' of 'OSError' objects>                                                    │
│             filename = <member 'filename' of 'OSError' objects>                                                 │
│            filename2 = <member 'filename2' of 'OSError' objects>                                                │
│           __module__ = 'requests.exceptions'                                                                    │
│             strerror = <member 'strerror' of 'OSError' objects>                                                 │
│ __suppress_context__ = <member '__suppress_context__' of 'BaseException' objects>                               │
│        __traceback__ = <attribute '__traceback__' of 'BaseException' objects>                                   │
│          __weakref__ = <attribute '__weakref__' of 'RequestException' objects>                                  │
│             add_note = def add_note(...)                                                                        │
│                        Exception.add_note(note) --                                                              │
│                        add a note to the exception                                                              │
│            __class__ = class __class__(...)                                                                     │
│                        type(object) -> the object's type                                                        │
│                        type(name, bases, dict, **kwds) -> a new type                                            │
│          __delattr__ = def __delattr__(self, name, /): Implement delattr(self, name).                           │
│              __dir__ = def __dir__(self, /): Default dir() implementation.                                      │
│               __eq__ = def __eq__(self, value, /): Ret

In [ ]:
# Adjust so that we have a list of URLs and we cycle through each of traditional, hybrid and electric.

In [9]:
path = Path.cwd()
path

PosixPath('/home/jsh/vehicle-co2')

In [10]:
# Declare file names
raw_path = path / 'data' / 'raw'
raw_file_name = raw_path / 'fuel-ratings-raw.csv'
merged_headers_file_name = raw_path / 'fuel-ratings-raw-headers-merged.csv'

In [11]:
extract_raw_data(url, raw_file_name)

Response status: 200
Content Type: text/csv
csv file: /home/jsh/vehicle-co2/data/raw/fuel-ratings-raw.csv saved


In [12]:
merge_top_two_rows(raw_file_name, merged_headers_file_name)

In [31]:
unprocessed_fuel_ratings_df = pd.read_csv(merged_headers_file_name)

/tmp/ipykernel_2874/1156222380.py:1: DtypeWarning: Columns (0,1,2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  unprocessed_fuel_ratings_df = pd.read_csv(merged_headers_file_name)


In [32]:
unprocessed_fuel_ratings_df.dropna(axis=1, how='all', inplace=True)

In [33]:
unprocessed_fuel_ratings_df.dropna(axis=0, thresh=10, inplace=True)

In [34]:
unprocessed_fuel_ratings_df

,Model Year,Make,Model,Vehicle Class,Engine Size (L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Hwy (L/100 km),Comb (L/100 km),Comb (mpg),CO2 Emissions (g/km),CO2 Rating,Smog Rating
0,2023,Acura,Integra,Full-size,1.5,4.0,AV7,Z,7.9,6.3,7.2,39.0,167.0,6.0,7.0
1,2023,Acura,Integra A-SPEC,Full-size,1.5,4.0,AV7,Z,8.1,6.5,7.4,38.0,172.0,6.0,7.0
2,2023,Acura,Integra A-SPEC,Full-size,1.5,4.0,M6,Z,8.9,6.5,7.8,36.0,181.0,6.0,6.0
3,2023,Acura,MDX SH-AWD,SUV: Small,3.5,6.0,AS10,Z,12.6,9.4,11.2,25.0,263.0,4.0,5.0
4,2023,Acura,MDX SH-AWD Type S,SUV: Standard,3.0,6.0,AS10,Z,13.8,11.2,12.4,23.0,291.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2023,Volvo,XC40 B5 AWD,SUV: Small,2.0,4.0,AS8,Z,10.2,7.9,9.2,31.0,215.0,5.0,5.0
829,2023,Volvo,XC60 B5 AWD,SUV: Small,2.0,4.0,AS8,Z,10.3,8.2,9.4,30.0,218.0,5.0,5.0
830,2023,Volvo,XC60 B6 AWD,SUV: Small,2.0,4.0,AS8,Z,11.1,8.7,10.0,28.0,233.0,5.0,7.0
831,2023,Volvo,XC90 B5 AWD,SUV: Standard,2.0,4.0,AS8,Z,10.5,8.4,9.6,29.0,223.0,5.0,5.0


In [39]:
rename_columns(unprocessed_fuel_ratings_df)
unprocessed_fuel_ratings_df

,model_year,make,model,vehicle_class,engine_size_(l),cylinders,transmission,fuel_type,fuel_consumption_city_(l/100_km),hwy_(l/100_km),comb_(l/100_km),comb_(mpg),co2_emissions_(g/km),co2_rating,smog_rating
0,2023,Acura,Integra,Full-size,1.5,4.0,AV7,Z,7.9,6.3,7.2,39.0,167.0,6.0,7.0
1,2023,Acura,Integra A-SPEC,Full-size,1.5,4.0,AV7,Z,8.1,6.5,7.4,38.0,172.0,6.0,7.0
2,2023,Acura,Integra A-SPEC,Full-size,1.5,4.0,M6,Z,8.9,6.5,7.8,36.0,181.0,6.0,6.0
3,2023,Acura,MDX SH-AWD,SUV: Small,3.5,6.0,AS10,Z,12.6,9.4,11.2,25.0,263.0,4.0,5.0
4,2023,Acura,MDX SH-AWD Type S,SUV: Standard,3.0,6.0,AS10,Z,13.8,11.2,12.4,23.0,291.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2023,Volvo,XC40 B5 AWD,SUV: Small,2.0,4.0,AS8,Z,10.2,7.9,9.2,31.0,215.0,5.0,5.0
829,2023,Volvo,XC60 B5 AWD,SUV: Small,2.0,4.0,AS8,Z,10.3,8.2,9.4,30.0,218.0,5.0,5.0
830,2023,Volvo,XC60 B6 AWD,SUV: Small,2.0,4.0,AS8,Z,11.1,8.7,10.0,28.0,233.0,5.0,7.0
831,2023,Volvo,XC90 B5 AWD,SUV: Standard,2.0,4.0,AS8,Z,10.5,8.4,9.6,29.0,223.0,5.0,5.0


In [42]:
unprocessed_fuel_ratings_df['make'].str.lower()

AttributeError: 'Series' object has no attribute 'lower'

In [ ]:
clean_fuel_ratings_df = clean_dataframe(unprocessed_fuel_ratings_df)

In [88]:
def extract_first_table(file_path):
    try:
        with open(file_path, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            rows = list(reader)

        # Find index of first empty row
        empty_row_idx = -1
        for i, row in enumerate(rows):
            if not any(row):
                empty_row_idx = i
                break
                
        if empty_row_idx >= 0:
        # Create a new CSV file with only the rows before the first empty row
            with open('test7.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rows[:empty_row_idx])
            print(f'Removed rows after the first empty row.')
        else:
            print(f'No empty row found in the file')
    except FileNotFoundError:
        print(f'File {file_path} not found.')
    except Exception as e:
        print('An error occured while processing the file:', e)

In [89]:
# extract_first_table(file_path)

Removed rows after the first empty row.


In [ ]:
# Export notebook to module.
from nbdev.export import nb_export
path = Path.cwd()
lib_path = path
nb_path = path / 'datadownload.ipynb'
nb_export(nb_path, lib_path)